In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

from tqdm.notebook import tqdm
from chainercv.evaluations import eval_detection_voc
from models import MIDN_Alexnet, MIDN_VGG16, OICR
from config import cfg
from datasets import VOCDectectionDataset
from torchvision.ops import roi_pool, nms
from sklearn.metrics import average_precision_score
from utils import *

In [2]:
year='2007'
pretrained='alexnet'
oicr = None
if pretrained == 'alexnet':
    midn = MIDN_Alexnet()
elif pretrained == 'vgg16':
    midn = MIDN_VGG16()
midn.to(cfg.DEVICE)

oicr = OICR(cfg.K)
oicr.to(cfg.DEVICE)

midn_checkpoints = torch.load(cfg.PATH.PT_PATH + "[F]Model_2007_alexnet.pt")
midn.load_state_dict(midn_checkpoints['midn_model_state_dict'])
# oicr.load_state_dict(midn_checkpoints['oicr_model_state_dict'])


testdata = VOCDectectionDataset("~/data/", year, 'test')
test_loader = data.DataLoader(testdata, cfg.TRAIN.BATCH_SIZE, shuffle=False)

log_file = cfg.PATH.LOG_PATH + f"Validate_{pretrained}" + ".txt"
write_log(log_file, f"model_name: OICR_{pretrained}")


total_pred_boxes = []
total_pred_labels = []
total_pred_scores = []
total_true_boxes = []
total_true_labels = []


KeyError: 'midn_model_state_dict'

In [4]:
with torch.no_grad():
    midn.eval()
    oicr.eval()
    for n_imgs, gt, n_regions, region in tqdm(test_loader, "Evaluation"):
        region = region.to(cfg.DEVICE)
#         avg_scores = torch.zeros((len(region[0]), 20), dtype=torch.float32)
#         for i in range(5):
#             per_img = n_imgs[i].to(cfg.DEVICE)
#             per_region = n_regions[i].to(cfg.DEVICE)
#             fc7, combined_scores = midn(per_img, per_region)
#             refine_scores = oicr(fc7)
#             for k in range(cfg.K):
#                 avg_scores += refine_scores[k][:, :20].cpu()
#             avg_scores += combined_scores.cpu()
#         avg_scores /= (5 * 4)
        
        savg_scores = torch.zeros((len(region[0]), 20), dtype=torch.float32)
        for i in range(5):
            per_img = n_imgs[i].to(cfg.DEVICE)
            per_region = n_regions[i].to(cfg.DEVICE)
            fc7, combined_scores = midn(per_img, per_region)
            for k in range(cfg.K):
                avg_scores += avg_scores.cpu()
        avg_scores /= (5 * 1)
        
    
        gt = gt.numpy()[0]
        gt_boxex = gt[:, :4]
        gt_labels = gt[:, -1]
    
        gt_labels_onehot = np.zeros(20)
        for label in gt_labels:
            gt_labels_onehot[int(label)] = 1

        per_pred_boxes = []
        per_pred_scores = []
        per_pred_labels = []
        
        region = region[0].cpu()
        
        for i in range(20):
            cls_scores = avg_scores[:, i]
            cls_region = region

            nms_filter = nms(cls_region, cls_scores, 0.3)
            per_pred_boxes.append(cls_region[nms_filter].numpy())
            per_pred_scores.append(cls_scores[nms_filter].numpy())
            per_pred_labels.append(np.full(len(nms_filter), i, dtype=np.int32))
            
        total_pred_boxes.append(np.concatenate(per_pred_boxes, axis=0))
        total_pred_scores.append(np.concatenate(per_pred_scores, axis=0))
        total_pred_labels.append(np.concatenate(per_pred_labels, axis=0))
        total_true_boxes.append(gt_boxex)
        total_true_labels.append(gt_labels)
        
    result = eval_detection_voc(
        total_pred_boxes,
        total_pred_labels,
        total_pred_scores,
        total_true_boxes,
        total_true_labels,
        iou_thresh=0.5,
        use_07_metric=True,
    )
    print(f"Avg AP: {result['ap']}")
    print(f"Avg mAP: {result['map']}")
#         write_log(log_file, f"Avg AP: {result['ap']}")
#         write_log(log_file, f"Avg mAP: {result['map']}")


Avg AP: [0.05681688 0.12065357 0.05569143 0.02008995 0.0305198  0.16428032
 0.1711747  0.23079644 0.003364   0.02530912 0.02825954 0.17148571
 0.10341195 0.147219   0.04436524 0.04192029 0.03184011 0.11029824
 0.19692285 0.03507965]
Avg mAP: 0.08947493939474795



Avg AP: [0.0380435  0.09586847 0.02967028 0.01421967 0.02488741 0.12029387
 0.12345616 0.14256291 0.00230058 0.09248682 0.09352449 0.11741401
 0.10741478 0.08556007 0.0268022  0.09133743 0.01528107 0.05298027
 0.13549627 0.03642921]
 
Avg mAP: 0.07230147397022946

Avg AP: [0.06553017 0.11218434 0.04370746 0.01982238 0.0250097  0.1920574
 0.184809   0.21933456 0.00361981 0.09866133 0.1002228  0.16470673
 0.13001323 0.14395161 0.04144958 0.09434894 0.02223967 0.06508374
 0.20686167 0.04100427]

Avg mAP: 0.09873091926663828